# 1. Import Library and Dataset

In [1]:
import pandas as pd
import numpy as np

from openai import OpenAI
import json

from tqdm import tqdm

In [2]:
dataset = pd.read_csv('dataset/search_keywords.csv')
print(dataset.head(5))

   id                       keyword       product product_category  brand
0   1                Apple iPad Pro        Tablet      Electronics  Apple
1   2             Canon DSLR camera   DSLR Camera      Electronics  Canon
2   3               wooden wardrobe      Wardrobe        Furniture    NaN
3   4  stainless steel kitchen sink  Kitchen Sink      Kitchenware    NaN
4   5           Asus ZenBook laptop        Laptop      Electronics   Asus


# 2. Making an API Request and Handling the Response

In [3]:
def load_prompt(file_path):
    with open(file_path, 'r') as file:
        return file.read()

In [4]:

def extract_data_from_keyword(keyword):
    # Load system content and user content
    system_content = load_prompt('prompts/search_keywords_system_prompt.txt')
    user_content = f"Keywords: {keyword}"

    # Initialize the OpenAI API client
    client = OpenAI()

    # Make a request to the OpenAI API to generate a chat completion
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_content
            },
            {
                "role": "user",
                "content": user_content
            }
        ],
        model="gpt-3.5-turbo",
        temperature=0.3,
        max_tokens=4096,
        top_p=0.8,
        response_format={ "type": "json_object" }
    )

    # Extract the completion result and token usage information from the response
    completion = chat_completion.choices[0].message.content
    result = json.loads(completion)
    prompt_tokens_used = chat_completion.usage.prompt_tokens
    completion_tokens_used = chat_completion.usage.completion_tokens

    return result, prompt_tokens_used, completion_tokens_used

# 3. Executing the Function for a Single Query

In [5]:
keyword = "Apple iPad Pro"
result, prompt_tokens_used, completion_tokens_used = extract_data_from_keyword(keyword)

print(f"Result : {result}")
print(f"Prompt Tokens Used : {prompt_tokens_used}")
print(f"Completion Tokens Used : {completion_tokens_used}")

Result : {'keyword': 'Apple iPad Pro', 'product': 'iPad Pro', 'product_category': 'Electronics', 'brand': 'Apple'}
Prompt Tokens Used : 238
Completion Tokens Used : 29


# 4. Executing the Function for a List of Queries

In [6]:
df_sample = dataset[['keyword']].head(5)

results = []
total_prompt_tokens_used = 0
total_completion_tokens_used = 0

for keyword in tqdm(df_sample['keyword'], desc="Processing keywords"):
    result, prompt_tokens_used, completion_tokens_used = extract_data_from_keyword(keyword)
    results.append(result)

    total_prompt_tokens_used += prompt_tokens_used
    total_completion_tokens_used += completion_tokens_used

df_result = pd.DataFrame(results)

Processing keywords: 100%|██████████| 5/5 [00:06<00:00,  1.30s/it]


In [7]:
print(f"Result:\n{df_result}")
print(f"Prompt Tokens Used : {total_prompt_tokens_used}")
print(f"Completion Tokens Used : {total_completion_tokens_used}")

Result:
                        keyword       product product_category  brand
0                Apple iPad Pro      iPad Pro      Electronics  Apple
1             Canon DSLR camera   DSLR Camera      Electronics  Canon
2               wooden wardrobe      Wardrobe        Furniture       
3  stainless steel kitchen sink  Kitchen Sink      Kitchenware       
4           Asus ZenBook laptop        Laptop      Electronics   Asus
Prompt Tokens Used : 1192
Completion Tokens Used : 156


# 5. Model Evaluation

In [8]:
df_sample = dataset[['keyword']]

results = []
total_prompt_tokens_used = 0
total_completion_tokens_used = 0

for keyword in tqdm(df_sample['keyword'], desc="Processing keywords"):
    result, prompt_tokens_used, completion_tokens_used = extract_data_from_keyword(keyword)
    results.append(result)

    total_prompt_tokens_used += prompt_tokens_used
    total_completion_tokens_used += completion_tokens_used

df_result = pd.DataFrame(results)

Processing keywords: 100%|██████████| 100/100 [02:04<00:00,  1.25s/it]


In [9]:
print(f"Result:\n{df_result}")
print(f"Prompt Tokens Used : {total_prompt_tokens_used}")
print(f"Completion Tokens Used : {total_completion_tokens_used}")

Result:
                         keyword           product product_category      brand
0                 Apple iPad Pro          iPad Pro      Electronics      Apple
1              Canon DSLR camera       DSLR Camera      Electronics      Canon
2                wooden wardrobe          Wardrobe        Furniture           
3   stainless steel kitchen sink      Kitchen Sink      Kitchenware           
4            Asus ZenBook laptop            Laptop      Electronics       Asus
..                           ...               ...              ...        ...
95           wooden dining table      Dining Table        Furniture           
96         Brother laser printer     Laser Printer      Electronics    Brother
97        Whirlpool refrigerator      Refrigerator  Home Appliances  Whirlpool
98         Adidas running shorts    Running Shorts         Clothing     Adidas
99          JBL portable speaker  Portable Speaker      Electronics        JBL

[100 rows x 4 columns]
Prompt Tokens Used :

In [10]:
merged_df = pd.merge(dataset, df_result, on='keyword', suffixes=('_actual', '_pred'))
merged_df = merged_df[merged_df['product_category_actual'] != 'Other Categories']

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate accuracy
accuracy = accuracy_score(merged_df['product_category_actual'], merged_df['product_category_pred'])
print(f'Accuracy: {accuracy}')

# Calculate precision, recall, and F1 score for each category
precision = precision_score(merged_df['product_category_actual'], merged_df['product_category_pred'], average=None, labels=merged_df['product_category_actual'].unique())
recall = recall_score(merged_df['product_category_actual'], merged_df['product_category_pred'], average=None, labels=merged_df['product_category_actual'].unique())
f1 = f1_score(merged_df['product_category_actual'], merged_df['product_category_pred'], average=None, labels=merged_df['product_category_actual'].unique())

# Create a DataFrame to display the metrics for each category
categories = merged_df['product_category_actual'].unique()
metrics_df = pd.DataFrame({
    'Category': categories,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
})

metrics_df['Precision'] = metrics_df['Precision'].apply(lambda x: f"{x:.2f}")
metrics_df['Recall'] = metrics_df['Recall'].apply(lambda x: f"{x:.2f}")
metrics_df['F1 Score'] = metrics_df['F1 Score'].apply(lambda x: f"{x:.2f}")

print(metrics_df)


Accuracy: 0.9518072289156626
          Category Precision Recall F1 Score
0      Electronics      1.00   0.97     0.98
1        Furniture      1.00   1.00     1.00
2      Kitchenware      1.00   0.83     0.91
3         Clothing      1.00   1.00     1.00
4  Home Appliances      0.75   1.00     0.86
